In [1]:
%%time
import os
import pandas as pd
import numpy as np

Wall time: 12.3 s


In [3]:
%%time
# Setting path 
# Get working directory
print(os.getcwd())

# Set working directory

os.chdir('C:\\Users\\ravigv\\Documents\\cosine_sim') 
print(os.getcwd())

C:\Users\ravigv\Documents\cosine_sim
C:\Users\ravigv\Documents\cosine_sim
Wall time: 0 ns


In [5]:
%%time
from gensim.models import Word2Vec 
import gensim.downloader as api


glove_model=None;
try:
    glove_model = gensim.models.KeyedVectors.load("./glovemodel.mod")
    print("Loaded glove model")
except:            
    glove_model = api.load('glove-twitter-25')
    glove_model.save("./glovemodel.mod")
    print("Saved glove model")
    
v2w_model=None;
try:
    v2w_model = gensim.models.KeyedVectors.load("./w2vecmodel.mod")
    print("Loaded w2v model")
except:            
    v2w_model = api.load('word2vec-google-news-300')
    v2w_model.save("./w2vecmodel.mod")
    print("Saved word2vec model")
    
    
fasttext_model300=None;
try:
    fasttext_model300 = gensim.models.KeyedVectors.load("./fasttext_model300.mod")
    print("Loaded fasttext_model300")
except:            
    fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')
    fasttext_model300.save("./fasttext_model300.mod")
    print("Saved fasttext_model300")

w2vec_embedding_size=len(v2w_model['computer']);
glove_embedding_size=len(glove_model['computer']);
fasttext_model300_size = len(fasttext_model300['computer']);



Saved glove model
Saved word2vec model
Saved fasttext_model300
Wall time: 9min 8s


In [6]:
%%time
df=pd.read_csv("ml_faq.csv");
df.columns=["questions","answers"];

df

Wall time: 799 ms


,questions,answers
0,What does the job hunting experience look like ?,Job hunting experience involves networking to ...
1,Any insights you can offer about the DS job ma...,"There are many kinds of roles, data scientist,..."
2,What’s the impact of Covid on hiring for DS ro...,Hiring is going to slow down. First in small c...
3,What skills and qualities do employers look fo...,The following are some skills employers usuall...
4,Do employers look for an advanced ML degree?,For more senior roles: People typically look f...
5,How does a typical day of a data scientist loo...,Here are some tasks in the typical day of a da...
6,Is preparation of algorithms and data structur...,Yes. In many data science interviews (ML Scien...
7,What is the mathematical background required t...,The following three are the basic building blo...
8,What are the various rounds in a data scientis...,Usually the data science interview has a subse...
9,What level of proficiency is needed for a data...,"Needs to be reasonably proficient. Again, a da..."


In [7]:
%%time
# Preprocessing

import re
import gensim 
from gensim.parsing.preprocessing import remove_stopwords

#from nltk.stem.lancaster import LancasterStemmer
#st = LancasterStemmer()

def clean_sentence(sentence, stopwords=False):
    
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    #sentence = re.sub(r'\s{2,}', ' ', sentence)
    
    if stopwords:
         sentence = remove_stopwords(sentence)
        
    return sentence

Wall time: 0 ns


In [8]:
%%time
def get_cleaned_sentences1(df, col, stopwords=False):
    
    cleaned_sentences=[]
    
    for index,row in df.iterrows():
#         print(index,row[col])
        cleaned=clean_sentence(row[col],stopwords);
        cleaned_sentences.append(cleaned);
    return cleaned_sentences;
        

        
# Calling the function get_cleaned_sentences

# Case1: Without stopwords
cleaned_sentences=get_cleaned_sentences1(df, 'questions', stopwords=True)
print(cleaned_sentences);

print("\n")

# Case2: With stopwords
cleaned_sentences_with_stopwords=get_cleaned_sentences1(df, 'questions', stopwords=False)
print(cleaned_sentences_with_stopwords);

['job hunting experience look like', 'insights offer ds job market', 'whats impact covid hiring ds roles', 'skills qualities employers look data scientist', 'employers look advanced ml degree', 'typical day data scientist look like', 'preparation algorithms data structures needed data science interview', 'mathematical background required data scientist', 'rounds data scientist interview', 'level proficiency needed data scientist coding']


['what does the job hunting experience look like ', 'any insights you can offer about the ds job market ', 'whats the impact of covid on hiring for ds roles', 'what skills and qualities do employers look for in a data scientist', 'do employers look for an advanced ml degree', 'how does a typical day of a data scientist look like', 'is preparation of algorithms and data structures needed for a data science interview ', 'what is the mathematical background required to be a data scientist ', 'what are the various rounds in a data scientist interview ', 

In [9]:
%%time
# Creating BOW

from gensim import corpora


sentences=cleaned_sentences_with_stopwords
#sentences=cleaned_sentences

# Split it by white space 
sentence_words = [[word for word in document.split() ]
         for document in sentences]

# Create corpora
from gensim import corpora
dictionary = corpora.Dictionary(sentence_words)
# for key, value in dictionary.items():
#     print(key, ' : ', value)


# Creating BOW

import pprint
bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]
for sent,embedding in zip(sentences,bow_corpus):
    print(sent)
    print(embedding)

what does the job hunting experience look like 
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
any insights you can offer about the ds job market 
[(3, 1), (6, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]
whats the impact of covid on hiring for ds roles
[(6, 1), (11, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)]
what skills and qualities do employers look for in a data scientist
[(5, 1), (7, 1), (17, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1)]
do employers look for an advanced ml degree
[(5, 1), (17, 1), (27, 1), (28, 1), (33, 1), (34, 1), (35, 1), (36, 1)]
how does a typical day of a data scientist look like
[(0, 1), (4, 1), (5, 1), (20, 1), (24, 2), (26, 1), (31, 1), (37, 1), (38, 1), (39, 1)]
is preparation of algorithms and data structures needed for a data science interview 
[(17, 1), (20, 1), (24, 1), (25, 1), (26, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), 

In [10]:
%%time
#question_orig="do I need to learn algorithms to be a data scientist ?";
query="What does a data scientist usually do ?"
query=clean_sentence(query,stopwords=False);
query_embedding = dictionary.doc2bow(query.split())


print("\n\n",query,"\n",query_embedding)



 what does a data scientist usually do  
 [(0, 1), (7, 1), (24, 1), (26, 1), (27, 1), (31, 1)]
Wall time: 0 ns


In [11]:
%%time
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

def get_cosim(query,docs):
      
    count = []
    simscore = []
    result = []

    for index,faq_embedding in enumerate(docs):
            sim=cosine_similarity(faq_embedding,query)[0][0]

            count.append(index)
            simscore.append(sim)
            result.append(sentences[index])



    dfScore = pd.DataFrame(list(zip(count, simscore, result)),
                  columns=['id','simscore', 'result'])


    dfsort = dfScore.sort_values('simscore', ascending = False)


    return dfsort


Wall time: 7.22 s


In [12]:
%%time
get_cosim(query_embedding,bow_corpus)

Wall time: 4.96 s


,id,simscore,result
0,0,1.000000,what does the job hunting experience look like
5,5,1.000000,how does a typical day of a data scientist loo...
1,1,0.316228,any insights you can offer about the ds job ma...
3,3,0.196116,what skills and qualities do employers look fo...
4,4,0.196116,do employers look for an advanced ml degree
2,2,0.164399,whats the impact of covid on hiring for ds roles
7,7,0.164399,what is the mathematical background required t...
8,8,0.164399,what are the various rounds in a data scientis...
9,9,0.141421,what level of proficiency is needed for a data...
6,6,0.058722,is preparation of algorithms and data structur...


In [13]:
%%time
# Getting Phrase Embeddings from Word Embeddings

def getWordVec(word,model):
        samp=model['computer'];
        vec=[0]*len(samp);
        try:
                vec=model[word];
        except:
                vec=[0]*len(samp);
        return (vec)


def getPhraseEmbedding(phrase,embeddingmodel):
                       
        samp=getWordVec('computer', embeddingmodel);
        vec=np.array([0]*len(samp));
        den=0;
        for word in phrase.split():
            #print(word)
            den=den+1;
            vec=vec+np.array(getWordVec(word,embeddingmodel));
        #vec=vec/den;
        #return (vec.tolist());
        return vec.reshape(1, -1)


Wall time: 0 ns


In [14]:
%%time
#With w2Vec model

sent_embeddings=[]
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,v2w_model))

question_embedding=getPhraseEmbedding(query,v2w_model)

get_cosim(question_embedding,sent_embeddings)

Wall time: 492 ms


,id,simscore,result
5,5,0.772894,how does a typical day of a data scientist loo...
7,7,0.644033,what is the mathematical background required t...
6,6,0.602005,is preparation of algorithms and data structur...
3,3,0.599192,what skills and qualities do employers look fo...
9,9,0.592059,what level of proficiency is needed for a data...
8,8,0.569657,what are the various rounds in a data scientis...
0,0,0.428834,what does the job hunting experience look like
1,1,0.339002,any insights you can offer about the ds job ma...
2,2,0.299255,whats the impact of covid on hiring for ds roles
4,4,0.283611,do employers look for an advanced ml degree


In [15]:
%%time
#With Glove model

sent_embeddings=[];
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,glove_model))

question_embedding=getPhraseEmbedding(query,glove_model)

get_cosim(question_embedding,sent_embeddings)

Wall time: 2.79 s


,id,simscore,result
5,5,0.954592,how does a typical day of a data scientist loo...
0,0,0.920684,what does the job hunting experience look like
3,3,0.876215,what skills and qualities do employers look fo...
2,2,0.857491,whats the impact of covid on hiring for ds roles
8,8,0.848120,what are the various rounds in a data scientis...
4,4,0.821041,do employers look for an advanced ml degree
6,6,0.800443,is preparation of algorithms and data structur...
7,7,0.793955,what is the mathematical background required t...
9,9,0.783340,what level of proficiency is needed for a data...
1,1,0.774017,any insights you can offer about the ds job ma...


In [16]:
%%time
#With fasttext model
sent_embeddings=[];
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,fasttext_model300))

question_embedding=getPhraseEmbedding(query,fasttext_model300)

get_cosim(question_embedding,sent_embeddings)

Wall time: 6.95 ms


,id,simscore,result
5,5,0.775747,how does a typical day of a data scientist loo...
0,0,0.737749,what does the job hunting experience look like
3,3,0.722098,what skills and qualities do employers look fo...
7,7,0.707121,what is the mathematical background required t...
9,9,0.703812,what level of proficiency is needed for a data...
6,6,0.694704,is preparation of algorithms and data structur...
1,1,0.674940,any insights you can offer about the ds job ma...
8,8,0.658610,what are the various rounds in a data scientis...
2,2,0.620278,whats the impact of covid on hiring for ds roles
4,4,0.583500,do employers look for an advanced ml degree
